# Notebook to build database on countries from a free API
This Notebook shows how to use a free API (no authorization or API key needed) to download some basic information about various countries around the world and put them in a DataFrame.

### Import libraries

In [55]:
import urllib.request, urllib.parse
from urllib.error import HTTPError,URLError
import pandas as pd

### Define the base URL

In [6]:
serviceurl = 'https://restcountries.eu/rest/v2/name/'

### Define function to pull the country data from the API

In [25]:
def get_country_data(country):
    country_name=country
    url = serviceurl + country_name
    
    try: 
        uh = urllib.request.urlopen(url)
    except HTTPError as e:
        print("Sorry! Could not retrive anything")
        return None
    except URLError as e:
        print('Failed to reach a server.')
        print('Reason: ', e.reason)
        return None
    else:
        data = uh.read().decode()
        print("Retrieved data on {}. Total {} characters read.".format(country_name,len(data)))
        return data

### Tests

In [26]:
country_name = 'Switzerland'

In [27]:
data=get_country_data(country_name)

Retrieved data on Switzerland. Total 1090 characters read.


In [28]:
country_name1 = 'Switzerland1'

In [29]:
data1=get_country_data(country_name1)

Sorry! Could not retrive anything


### Use the built-in `JSON` library to read the data properly

In [13]:
import json

In [14]:
# Load from string 'data'
x=json.loads(data)

In [15]:
# Load the only element
y=x[0]

In [16]:
type(y)

dict

In [17]:
y.keys()

dict_keys(['name', 'topLevelDomain', 'alpha2Code', 'alpha3Code', 'callingCodes', 'capital', 'altSpellings', 'region', 'subregion', 'population', 'latlng', 'demonym', 'area', 'gini', 'timezones', 'borders', 'nativeName', 'numericCode', 'currencies', 'languages', 'translations', 'flag', 'regionalBlocs', 'cioc'])

In [19]:
for k,v in y.items():
    print(f"{k}: {v}")

name: Switzerland
topLevelDomain: ['.ch']
alpha2Code: CH
alpha3Code: CHE
callingCodes: ['41']
capital: Bern
altSpellings: ['CH', 'Swiss Confederation', 'Schweiz', 'Suisse', 'Svizzera', 'Svizra']
region: Europe
subregion: Western Europe
population: 8341600
latlng: [47.0, 8.0]
demonym: Swiss
area: 41284.0
gini: 33.7
timezones: ['UTC+01:00']
borders: ['AUT', 'FRA', 'ITA', 'LIE', 'DEU']
nativeName: Schweiz
numericCode: 756
currencies: [{'code': 'CHF', 'name': 'Swiss franc', 'symbol': 'Fr'}]
languages: [{'iso639_1': 'de', 'iso639_2': 'deu', 'name': 'German', 'nativeName': 'Deutsch'}, {'iso639_1': 'fr', 'iso639_2': 'fra', 'name': 'French', 'nativeName': 'français'}, {'iso639_1': 'it', 'iso639_2': 'ita', 'name': 'Italian', 'nativeName': 'Italiano'}]
translations: {'de': 'Schweiz', 'es': 'Suiza', 'fr': 'Suisse', 'ja': 'スイス', 'it': 'Svizzera', 'br': 'Suíça', 'pt': 'Suíça', 'nl': 'Zwitserland', 'hr': 'Švicarska', 'fa': 'سوئیس'}
flag: https://restcountries.eu/data/che.svg
regionalBlocs: [{'acrony

In [20]:
for i in y['languages']:
    print(i['name'])

German
French
Italian


### Define a function which can take a list of countries and return a DataFrame containing key info
* Capital
* Region
* Sub-region
* Population
* lattitude/longitude
* Area
* Gini index
* Timezones
* Currencies
* Languages

In [75]:
def build_country_database(list_country):
    """
    """
    import pandas as pd
    import json
    # Define an empty dictionary with keys
    country_dict={'Country':[],'Capital':[],'Region':[],'Sub-region':[],'Population':[],
                  'Lattitude':[],'Longitude':[],'Area':[],'Gini':[],'Timezones':[],
                  'Currencies':[],'Languages':[]}
    
    for c in list_country:
        data = get_country_data(c)
        if data!=None:
            x = json.loads(data)
            y=x[0]
            country_dict['Country'].append(y['name'])
            country_dict['Capital'].append(y['capital'])
            country_dict['Region'].append(y['region'])
            country_dict['Sub-region'].append(y['subregion'])
            country_dict['Population'].append(y['population'])
            country_dict['Lattitude'].append(y['latlng'][0])
            country_dict['Longitude'].append(y['latlng'][1])
            country_dict['Area'].append(y['area'])
            country_dict['Gini'].append(y['gini'])
            # Note the code to handle possibility of multiple timezones as a list
            if len(y['timezones'])>1:
                country_dict['Timezones'].append(','.join(y['timezones']))
            else:
                country_dict['Timezones'].append(y['timezones'][0])
            # Note the code to handle possibility of multiple currencies as dictionaries
            if len(y['currencies'])>1:
                lst_currencies = []
                for i in y['currencies']:
                    lst_currencies.append(i['name'])
                country_dict['Currencies'].append(','.join(lst_currencies))
            else:
                country_dict['Currencies'].append(y['currencies'][0]['name'])
            # Note the code to handle possibility of multiple languages as dictionaries
            if len(y['languages'])>1:
                lst_languages = []
                for i in y['languages']:
                    lst_languages.append(i['name'])
                country_dict['Languages'].append(','.join(lst_languages))
            else:
                country_dict['Languages'].append(y['languages'][0]['name'])
    
    # Return as a Pandas DataFrame
    return pd.DataFrame(country_dict)

In [76]:
df1=build_country_database(['Nigeria','Switzerland','France','Russia','Kenya','Singapore'])

Retrieved data on Nigeria. Total 1004 characters read.
Retrieved data on Switzerland. Total 1090 characters read.
Retrieved data on France. Total 1047 characters read.
Retrieved data on Russia. Total 1120 characters read.
Retrieved data on Kenya. Total 1052 characters read.
Retrieved data on Singapore. Total 1223 characters read.


In [77]:
df1

,Country,Capital,Region,Sub-region,Population,Lattitude,Longitude,Area,Gini,Timezones,Currencies,Languages
0,Nigeria,Abuja,Africa,Western Africa,186988000,10.000000,8.0,923768.0,48.8,UTC+01:00,Nigerian naira,English
1,Switzerland,Bern,Europe,Western Europe,8341600,47.000000,8.0,41284.0,33.7,UTC+01:00,Swiss franc,"German,French,Italian"
2,France,Paris,Europe,Western Europe,66710000,46.000000,2.0,640679.0,32.7,"UTC-10:00,UTC-09:30,UTC-09:00,UTC-08:00,UTC-04...",Euro,French
3,Russian Federation,Moscow,Europe,Eastern Europe,146599183,60.000000,100.0,17124442.0,40.1,"UTC+03:00,UTC+04:00,UTC+06:00,UTC+07:00,UTC+08...",Russian ruble,Russian
4,Kenya,Nairobi,Africa,Eastern Africa,47251000,1.000000,38.0,580367.0,47.7,UTC+03:00,Kenyan shilling,"English,Swahili"
5,Singapore,Singapore,Asia,South-Eastern Asia,5535000,1.366667,103.8,710.0,48.1,UTC+08:00,"Brunei dollar,Singapore dollar","English,Malay,Tamil,Chinese"
